In [1]:
import pickle
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.cluster import KMeans

# Charge les données et objets pré-entraînés
with open('model/df.pkl', 'rb') as f:
    df = pickle.load(f)
    
def load_model_objects():
    with open('model/vectorizer.pkl', 'rb') as f:
        vectorizer = pickle.load(f)
    with open('model/svd.pkl', 'rb') as f:
        svd = pickle.load(f)
    with open('model/top_tags.pkl', 'rb') as f:
        top_tags = pickle.load(f)
    with open('model/X_use_embeddings.pkl', 'rb') as f:
        X_use_np = pickle.load(f)
    with open('model/y.pkl', 'rb') as f:
        y = pickle.load(f)
    return vectorizer, svd, top_tags, X_use_np, y

vectorizer, svd, top_tags, X_use_np, y = load_model_objects()

# Modèle BERT et le tokenizer depuis Hugging Face
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')


c:\Users\leenc\Documents\openclassrooms\p5\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\leenc\Documents\openclassrooms\p5\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [2]:
print(df.head())
print(df.columns)


                                                   Title  \
33553                   Convert PDF to .docx with Python   
9427   Unknown Option error from Babel in React-Nativ...   
199                           Not Found for an API route   
12447                          Remove emojis from string   
39489  Permission denied when installing npm modules ...   

                                                    Body  \
33553  <p>I'm trying to find a way to convert a PDF f...   
9427   <p>I am building a react-native app with types...   
199    <p>In my Next.js project I have <code>main/app...   
12447  <p>I'm using imagettftext function on my code ...   
39489  <p>I'm trying to install <code>node-g.raphael<...   

                                                    Tags        Id  Score  \
33553              [python, pdf, docx, libreoffice, doc]  49965818      5   
9427   [javascript, typescript, react-native, babeljs...  64225453     27   
199                                [javascript,

In [3]:

# Modèle BERT et le tokenizer depuis Hugging Face
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')



KMeans(n_clusters=10, random_state=42)

In [4]:
# Fonction pour transformer un texte en embeddings BERT
def transform_text_to_embedding(text):
    inputs = tokenizer(text, return_tensors='tf', truncation=True, padding=True, max_length=128)
    outputs = bert_model(inputs)
    # Moyenne des vecteurs de chaque mot pour créer un seul vecteur pour le texte
    embeddings = tf.reduce_mean(outputs.last_hidden_state, axis=1).numpy()
    return embeddings

# Générer les embeddings pour tout le corpus
embeddings = np.vstack(df['Cleaned_Body'].apply(transform_text_to_embedding).values)

# Algorithme de clustering (KMeans)
n_clusters = 10  # Nombre de clusters à définir
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Entraîner KMeans sur les embeddings BERT
kmeans.fit(embeddings)

In [5]:


# Tags basés sur le clustering
def suggest_tags(text):
    # Transforme le texte en embedding
    embedding = transform_text_to_embedding(text)
    
    # Cluster le plus proche
    cluster = kmeans.predict(embedding)
    
    # Eléments dans ce cluster
    cluster_indices = np.where(kmeans.labels_ == cluster)[0]
    
    # Fréquence des tags dans ce cluster
    tag_counts = {}
    for idx in cluster_indices:
        tags = y[idx]  # Liste de tag
        for i, tag_active in enumerate(tags):
            if tag_active:  # Si le tag est actif (1)
                tag_name = top_tags[i]
                if tag_name in tag_counts:
                    tag_counts[tag_name] += 1
                else:
                    tag_counts[tag_name] = 1
    
    # Trie les tags par fréquence et retourne les plus communs
    sorted_tags = sorted(tag_counts.items(), key=lambda item: item[1], reverse=True)
    suggested_tags = [tag for tag, count in sorted_tags[:5]]  # Limite : 5 suggestions
    
    return suggested_tags

# Test avec une question
new_question = "how to center a div?"
suggested_tags = suggest_tags(new_question)

print(f"Suggested tags for '{new_question}': {suggested_tags}")

ValueError: X has 768 features, but KMeans is expecting 12086 features as input.